In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Подготовка данных
df_plot = va_df_filtered_dop.groupby(["city_model", "selected_type_house"]).size().reset_index(name="count")

# Визуализация
plt.figure(figsize=(14, 7))
sns.set(style="whitegrid")

ax = sns.barplot(
    data=df_plot,
    x="city_model",
    y="count",
    hue="selected_type_house",
    palette="Set2"
)

# Оформление
plt.title("📊 Распределение типов домов по городской модели", fontsize=16)
plt.xlabel("Городская модель", fontsize=12)
plt.ylabel("Количество участков", fontsize=12)
plt.xticks(rotation=15)
plt.legend(title="Тип дома", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
# Распределение типов домов в каждой городской модели
type_distribution = va_df_filtered_0.groupby(["city_model", "selected_type_house"]).size().unstack(fill_value=0)

print("\n📊 Распределение типов домов по моделям:")
print(type_distribution)
type_distribution.to_excel(r"D:\STUDY\ВКР\VKR_CODE\result_3\аналитика_5.xlsx")

In [ ]:
from IPython.display import display
import pandas as pd

# Функция сбора нужной статистики (без медианы, минимума и максимума)
def score_stats(df):
    return {
        "Кол-во объектов": len(df),
        "Среднее": round(df["total_score"].mean(), 2),
        "Ст. отклонение": round(df["total_score"].std(), 2),
        "Кол-во > 0": (df["total_score"] > 0).sum(),
        "Кол-во < 0": (df["total_score"] < 0).sum(),
        "Кол-во = 0": (df["total_score"] == 0).sum()
    }

# Получаем статистику
dop_stats = score_stats(zones_dop)
zero_stats = score_stats(zones_0)

# Собираем таблицу
df_stats = pd.DataFrame({
    "zones_dop": dop_stats,
    "zones_0": zero_stats
})
df_stats["Разница (dop - 0)"] = df_stats["zones_dop"] - df_stats["zones_0"]

# Красивый вывод
styled = df_stats.style.set_table_styles(
    [{'selector': 'th', 'props': [('border', '1px solid black'), ('padding', '6px')]},
     {'selector': 'td', 'props': [('border', '1px solid black'), ('padding', '6px')]}]
).set_caption("Сравнительная аналитика total_score (без медианы/минимума/максимума)")

display(styled)


In [ ]:
import pandas as pd
import geopandas as gpd

# Уникальные типы городской среды
city_models = zones_0['city_model'].unique()

# Словарь для сбора всех данных
results = []

# Анализ по каждой модели городской среды
for model in city_models:
    model_data = zones_0[zones_0['city_model'] == model]
    va_data = va_for_house_0[va_for_house_0['city_model'] == model]
    select_data = select_va_0[select_va_0['city_model'] == model]

    result = {
        # --- Информация о населении ---
        'Total Population (sum_population)': model_data['sum_population'].sum(),
        'Average New Population': model_data['new_population'].mean(),
        'Average New Population (dop)': model_data['new_population_dop'].mean(),
        'Average Population Density': model_data['density_population'].mean(),
        'Zones with Additional Population': (model_data['new_population'] > 0).sum(),
        'Average Number of Floors': model_data['avg_number_of_floors'].mean(),

        # --- Пространственная аналитика ---
        'Total Zones': model_data.shape[0],
        'Average Area of Zone': model_data['area_zone'].mean(),
        'Median Area of Zone': model_data['area_zone'].median(),
        'Average Living Area': model_data['sum_living_area'].mean(),
        'Zones with Green Above Norm': (model_data['difference_from_normative'] > 0).sum(),
        'Zones with Population Deficit': (model_data['deficit_density'] > 0).sum(),

        # --- Коэффициент застроенности ---
        'Average Building Density': (model_data['sum_footprint_area'] / model_data['area_zone']).mean(),  # Коэффициент застроенности

        # --- Социальные сервисы ---
        'Average Polyclinic Free Places': model_data['polyclinic_free_places'].mean(),
        'Average Polyclinic Employed Places': model_data['polyclinic_employed_places'].mean(),
        'Average School Free Places': model_data['school_free_places'].mean(),
        'Average School Employed Places': model_data['school_employed_places'].mean(),
        'Average Kindergarten Free Places': model_data['kindergarten_free_places'].mean(),
        'Average Kindergarten Employed Places': model_data['kindergarten_employed_places'].mean(),
        'Average Green Per Capita': model_data['green_per_capita'].mean(),
        'Zones without School Places': (model_data['school_free_places'] == 0).sum(),
        'Zones without Kindergarten Places': (model_data['kindergarten_free_places'] == 0).sum(),
        'Zones without Polyclinic Places': (model_data['polyclinic_free_places'] == 0).sum(),
        'Average School Coverage': model_data.groupby('school_id_service').size().mean(),
        'Average Kindergarten Coverage': model_data.groupby('kindergarten_id_service').size().mean(),
        'Average Polyclinic Coverage': model_data.groupby('polyclinic_id_service').size().mean(),
        'Zones with Schools': model_data[model_data['school_id_service'].notna()].shape[0],
        'Zones with Kindergartens': model_data[model_data['kindergarten_id_service'].notna()].shape[0],
        'Zones with Polyclinics': model_data[model_data['polyclinic_id_service'].notna()].shape[0],

        # --- Участки VA ---
        'Average Area of VA': va_data['area_va'].mean(),
        'Median Area of VA': va_data['area_va'].median(),
        'Total VA Count': va_data.shape[0],

        # --- Участки Select_VA ---
        'Total Select VA Parcels': select_data.shape[0],
        'Total New Population on Select VA': select_data['new_population'].sum(),

        # --- Зоны, которым не хватает школ, детсадов и поликлиник ---
        'Zones Lacking Schools': (model_data['need_dop_service'] == 'school').sum(),
        'Zones Lacking Kindergartens': (model_data['need_dop_service'] == 'kindergarten').sum(),
        'Zones Lacking Polyclinics': (model_data['need_dop_service'] == 'polyclinic').sum(),

        # --- Зоны с необходимыми местами в сервисах ---
        'Zones with All Necessary Services': (model_data['new_population'] > 0).sum(),
    }

    # Добавляем тип городской модели
    result['City Model'] = model
    results.append(result)

# Преобразуем в DataFrame
df = pd.DataFrame(results)

# Названия на русском
column_rus = {
    'Total Population (sum_population)': 'Общее население',
    'Average New Population': 'Среднее новое население',
    'Average New Population (dop)': 'Среднее новое население (доп)',
    'Average Population Density': 'Средняя плотность населения',
    'Zones with Additional Population': 'Зоны с новым населением',
    'Average Number of Floors': 'Средняя этажность',
    'Total Zones': 'Всего зон',
    'Average Area of Zone': 'Средняя площадь зоны',
    'Median Area of Zone': 'Медианная площадь зоны',
    'Average Living Area': 'Средняя жилая площадь',
    'Zones with Green Above Norm': 'Зоны с озеленением выше нормы',
    'Zones with Population Deficit': 'Зоны с дефицитом населения',
    'Average Building Density': 'Средняя плотность застройки',
    'Average Polyclinic Free Places': 'Свободные места в поликлиниках (среднее)',
    'Average Polyclinic Employed Places': 'Занятые места в поликлиниках (среднее)',
    'Average School Free Places': 'Свободные места в школах (среднее)',
    'Average School Employed Places': 'Занятые места в школах (среднее)',
    'Average Kindergarten Free Places': 'Свободные места в садах (среднее)',
    'Average Kindergarten Employed Places': 'Занятые места в садах (среднее)',
    'Average Green Per Capita': 'Озеленение на душу населения',
    'Zones without School Places': 'Зоны без мест в школах',
    'Zones without Kindergarten Places': 'Зоны без мест в садах',
    'Zones without Polyclinic Places': 'Зоны без мест в поликлиниках',
    'Average School Coverage': 'Средняя обеспеченность школами',
    'Average Kindergarten Coverage': 'Средняя обеспеченность садами',
    'Average Polyclinic Coverage': 'Средняя обеспеченность поликлиниками',
    'Zones with Schools': 'Зоны со школами',
    'Zones with Kindergartens': 'Зоны с садами',
    'Zones with Polyclinics': 'Зоны с поликлиниками',
    'Average Area of VA': 'Средняя площадь VA',
    'Median Area of VA': 'Медианная площадь VA',
    'Total VA Count': 'Всего VA',
    'Total Select VA Parcels': 'Всего участков под застройку',
    'Total New Population on Select VA': 'Потенциальное население на участках',
    'Zones Lacking Schools': 'Зоны, которым не хватает школ',
    'Zones Lacking Kindergartens': 'Зоны, которым не хватает детсадов',
    'Zones Lacking Polyclinics': 'Зоны, которым не хватает поликлиник',
    'Zones with All Necessary Services': 'Зоны с необходимыми местами в сервисах',
}

# Единицы измерения
column_units = {
    'Total Population (sum_population)': 'чел.',
    'Average New Population': 'чел.',
    'Average New Population (dop)': 'чел.',
    'Average Population Density': 'чел./га',
    'Zones with Additional Population': 'ед.',
    'Average Number of Floors': 'этажей',
    'Total Zones': 'ед.',
    'Average Area of Zone': 'м²',
    'Median Area of Zone': 'м²',
    'Average Living Area': 'м²',
    'Zones with Green Above Norm': 'ед.',
    'Zones with Population Deficit': 'ед.',
    'Average Building Density': '',
    'Average Polyclinic Free Places': 'мест',
    'Average Polyclinic Employed Places': 'мест',
    'Average School Free Places': 'мест',
    'Average School Employed Places': 'мест',
    'Average Kindergarten Free Places': 'мест',
    'Average Kindergarten Employed Places': 'мест',
    'Average Green Per Capita': 'м²/чел.',
    'Zones without School Places': 'ед.',
    'Zones without Kindergarten Places': 'ед.',
    'Zones without Polyclinic Places': 'ед.',
    'Average School Coverage': 'зон/школу',
    'Average Kindergarten Coverage': 'зон/сад',
    'Average Polyclinic Coverage': 'зон/поликлинику',
    'Zones with Schools': 'ед.',
    'Zones with Kindergartens': 'ед.',
    'Zones with Polyclinics': 'ед.',
    'Average Area of VA': 'м²',
    'Median Area of VA': 'м²',
    'Total VA Count': 'ед.',
    'Total Select VA Parcels': 'ед.',
    'Total New Population on Select VA': 'чел.',
    'Zones Lacking Schools': 'ед.',
    'Zones Lacking Kindergartens': 'ед.',
    'Zones Lacking Polyclinics': 'ед.',
    'Zones with All Necessary Services': 'ед.',
}

# Переупорядочим колонки, добавим столбцы на русском и с единицами измерения
df_transposed = df.set_index('City Model').transpose()
df_transposed['Russian Name'] = df_transposed.index.map(column_rus)
df_transposed['Units'] = df_transposed.index.map(column_units)

# Переставим порядок столбцов
df_transposed = df_transposed[['Russian Name', 'Units'] + [col for col in df_transposed.columns if col not in ['Russian Name', 'Units']]]

# Сохраняем в Excel
df_transposed.to_excel(r"D:\STUDY\ВКР\VKR_CODE\result_3\аналитика_5.xlsx")


In [ ]:
# 1. Общие значения city_model
common_city_models = sorted(set(zones_dop["city_model"]).intersection(set(zones_0["city_model"])))

rows = []

# 2. По всем общим city_model
for model in common_city_models:
    dop_filtered = zones_dop[zones_dop["city_model"] == model]
    zero_filtered = zones_0[zones_0["city_model"] == model]

    dop_stats = score_stats(dop_filtered)
    zero_stats = score_stats(zero_filtered)

    for key in dop_stats.keys():
        rows.append({
            "city_model": model,
            "Показатель": key,
            "zones_dop": dop_stats[key],
            "zones_0": zero_stats[key],
            "Разница (dop - 0)": dop_stats[key] - zero_stats[key]
        })

# 3. Добавим строку "Итого" (общая оценка по всем моделям)
total_dop_stats = score_stats(zones_dop[zones_dop["city_model"].isin(common_city_models)])
total_zero_stats = score_stats(zones_0[zones_0["city_model"].isin(common_city_models)])

for key in total_dop_stats.keys():
    rows.append({
        "city_model": "Итого",
        "Показатель": key,
        "zones_dop": total_dop_stats[key],
        "zones_0": total_zero_stats[key],
        "Разница (dop - 0)": total_dop_stats[key] - total_zero_stats[key]
    })

# 4. Финальный DataFrame
df_all_stats = pd.DataFrame(rows)

# 5. Форматируем для читаемости
df_pivoted = df_all_stats.pivot(index=["city_model", "Показатель"], columns=[], values=["zones_dop", "zones_0", "Разница (dop - 0)"])

# 6. Красивый вывод
styled_all = df_pivoted.style.set_table_styles(
    [{'selector': 'th', 'props': [('border', '1px solid black'), ('padding', '6px')]},
     {'selector': 'td', 'props': [('border', '1px solid black'), ('padding', '6px')]}]
).set_caption("Сравнительная аналитика total_score по city_model с общей строкой")

display(styled_all)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def city_model_stats(df):
    grouped = df.groupby("city_model")["total_score"]
    return pd.DataFrame({
        "Кол-во объектов": grouped.count(),
        "Среднее": grouped.mean().round(2),
        "Ст. отклонение": grouped.std().round(2),
        "Кол-во > 0": grouped.apply(lambda x: (x > 0).sum()),
        "Кол-во = 0": grouped.apply(lambda x: (x == 0).sum()),
        "Кол-во < 0": grouped.apply(lambda x: (x < 0).sum())
    }).reset_index()

dop_stats = city_model_stats(zones_dop)
zero_stats = city_model_stats(zones_0)
df = pd.merge(dop_stats, zero_stats, on="city_model", suffixes=("_dop", "_0"))

def plot_bars_summary(df):
    x = np.arange(len(df['city_model']))
    width = 0.2

    fig, axs = plt.subplots(2, 1, figsize=(14, 11), sharex=True, gridspec_kw={'hspace': 0.4})
    bar_kwargs = dict(align='center')

    # === Верхний график: Кол-во > 0 и < 0 ===
    bars = [
        axs[0].bar(x - width*1.5, df['Кол-во > 0_dop'], width, label='dop > 0', color='#D97706'),
        axs[0].bar(x - width/2, df['Кол-во < 0_dop'], width, label='dop < 0', color='#92400E'),
        axs[0].bar(x + width/2, df['Кол-во > 0_0'], width, label='0 > 0', color='#38B2AC'),
        axs[0].bar(x + width*1.5, df['Кол-во < 0_0'], width, label='0 < 0', color='#2C7A7B')
    ]

    axs[0].set_ylabel('Количество')
    axs[0].set_title('Распределение положительных и отрицательных total_score по city_model')
    axs[0].legend()
    axs[0].grid(True, linestyle='--', alpha=0.5)

    # Подписи значений
    for group in bars:
        for bar in group:
            height = bar.get_height()
            axs[0].annotate(f'{int(height)}',
                            xy=(bar.get_x() + bar.get_width() / 2, height),
                            xytext=(0, 3),
                            textcoords="offset points",
                            ha='center', va='bottom', fontsize=8)

    # === Разделительная линия ===
    axs[0].axhline(0, color='gray', linewidth=0.8)

    # === Нижний график: Средние значения ===
    bars_mean = [
        axs[1].bar(x - width/2, df['Среднее_dop'], width, label='dop (среднее)', color='#D97706'),
        axs[1].bar(x + width/2, df['Среднее_0'], width, label='0 (среднее)', color='#38B2AC')
    ]

    axs[1].set_ylabel('Среднее значение')
    axs[1].set_title('Среднее total_score по city_model')
    axs[1].set_xticks(x)
    axs[1].set_xticklabels(df['city_model'], rotation=45, ha='right')
    axs[1].legend()
    axs[1].grid(True, linestyle='--', alpha=0.5)

    for group in bars_mean:
        for bar in group:
            height = bar.get_height()
            axs[1].annotate(f'{height:.2f}',
                            xy=(bar.get_x() + bar.get_width() / 2, height),
                            xytext=(0, 3),
                            textcoords="offset points",
                            ha='center', va='bottom', fontsize=8)

    plt.tight_layout()
    plt.show()

# Запуск
plot_bars_summary(df)



In [ ]:
import pandas as pd

def city_model_summary_stats(df):
    # Группировка по city_model и вычисление статистик
    grouped = df.groupby("city_model")["total_score"]
    
    summary = pd.DataFrame({
        "Количество зон с отрицательной оценкой": grouped.apply(lambda x: (x < 0).sum()),
        "Количество зон с положительной оценкой": grouped.apply(lambda x: (x > 0).sum()),
        "Медианное значение": grouped.median()  # Обратите внимание, что столбец будет называться просто "Медианное значение"
    }).reset_index()

    return summary

# Расчет статистик для zones_0 и zones_dop
zones_0_stats = city_model_summary_stats(zones_0)
zones_dop_stats = city_model_summary_stats(zones_dop)

# Мерджим таблицы
df_stats = pd.merge(zones_0_stats, zones_dop_stats, on="city_model", suffixes=('_zones_0', '_zones_dop'))

# Создаем итоговую таблицу
final_table = pd.DataFrame({
    "parameters": [
        "Общее количество зон с отрицательной оценкой в zones_0",
        "Общее количество зон с отрицательной оценкой в zones_dop",
        "Общее количество зон с положительной оценкой в zones_0",
        "Общее количество зон с положительной оценкой в zones_dop",
        "Медианное значение оценки в zones_0",
        "Медианное значение оценки в zones_dop"
    ],
    "low_rise": [
        df_stats[df_stats["city_model"] == "low_rise"]["Количество зон с отрицательной оценкой_zones_0"].sum(),
        df_stats[df_stats["city_model"] == "low_rise"]["Количество зон с отрицательной оценкой_zones_dop"].sum(),
        df_stats[df_stats["city_model"] == "low_rise"]["Количество зон с положительной оценкой_zones_0"].sum(),
        df_stats[df_stats["city_model"] == "low_rise"]["Количество зон с положительной оценкой_zones_dop"].sum(),
        df_stats[df_stats["city_model"] == "low_rise"]["Медианное значение_zones_0"].values[0],
        df_stats[df_stats["city_model"] == "low_rise"]["Медианное значение_zones_dop"].values[0],
    ],
    "medium": [
        df_stats[df_stats["city_model"] == "medium"]["Количество зон с отрицательной оценкой_zones_0"].sum(),
        df_stats[df_stats["city_model"] == "medium"]["Количество зон с отрицательной оценкой_zones_dop"].sum(),
        df_stats[df_stats["city_model"] == "medium"]["Количество зон с положительной оценкой_zones_0"].sum(),
        df_stats[df_stats["city_model"] == "medium"]["Количество зон с положительной оценкой_zones_dop"].sum(),
        df_stats[df_stats["city_model"] == "medium"]["Медианное значение_zones_0"].values[0],
        df_stats[df_stats["city_model"] == "medium"]["Медианное значение_zones_dop"].values[0],
    ],
    "central": [
        df_stats[df_stats["city_model"] == "central"]["Количество зон с отрицательной оценкой_zones_0"].sum(),
        df_stats[df_stats["city_model"] == "central"]["Количество зон с отрицательной оценкой_zones_dop"].sum(),
        df_stats[df_stats["city_model"] == "central"]["Количество зон с положительной оценкой_zones_0"].sum(),
        df_stats[df_stats["city_model"] == "central"]["Количество зон с положительной оценкой_zones_dop"].sum(),
        df_stats[df_stats["city_model"] == "central"]["Медианное значение_zones_0"].values[0],
        df_stats[df_stats["city_model"] == "central"]["Медианное значение_zones_dop"].values[0],
    ]
})

# Экспортируем таблицу в Excel
final_table.to_excel(r"D:\STUDY\ВКР\VKR_CODE\result_3\аналитика_2.xlsx", index=False)

# Покажем финальную таблицу
final_table

